## Meta

#### Settings:


In [11]:
start_over = False                           # True will overwrite saved models with new models / False to Lock settings here.
amount_of_symbols_per_action_sample = 3      # Symbols to propose per coin.
aggregate_loss_history_epochs = 30           # Calculate aggregated loss based on last X epochs
minimum_prediction_prob_trade_NextIsNone = 0.33      # Minimum 33% prediciton probability before taking action if Next option is None (out of 4 options)
minimum_prediction_prob_trade_NextIsNotNone = 0.29   # Minimum 29% prediciton probability before taking action if Next option is another symbol (out of 4 options)
binary_target_minimum_increase_min = 0.0005  #                 Minimum 0.05 % increase.
binary_target_minimum_increase_max = 0.002   # Starting with more than 0.2  % increase.
seconds_inbetween = 9                        # Seconds between handling training data.
seconds_inbetween_pibeline = 3               # Seconds between cycling the pibeline.

pred_epoch = 30                              # =  4.5  min    | Predict in how many epochs.
interval_columns = 150                       # = 22.5  min    | Amount of epochs in history per sample.

catboost_iterations = 7
catboost_depth = 10
catboost_learning_rate = 0.10
catboost_one_hot_max_size = 0
catboost_ctr_target_border_count = 50
catboost_model_size_reg = 0
catboost_max_ctr_complexity = 0
catboost_l2_leaf_reg = None
catboost_min_data_in_leaf = None
catboost_random_strength = None
catboost_bootstrap_type = 'Bernoulli'
catboost_subsample = 0.5
catboost_bagging_temperature = None

is_google_colab, is_google_cloud = False, True
datasets_dir = 'datasets12'

real_wallet_dont_use_euro = 20
do_real_money_trade = True                       # prod: True
do_training = True                               # prod: True
do_gather_dataset = False                        # prod: False
model_name_for_trading = 'CatBoostAction'
debug_symbol = 'SUNUSDT'
use_models = "NN" # Neural Network

### load_models:

In [12]:
def load_models():
    global models

    if not exists(root_dir):
        os.mkdir(root_dir)
    if not exists(root_dir + '/' + datasets_dir):
        os.mkdir(root_dir + '/' + datasets_dir)
    if not exists(root_dir + '/catboost'):
        os.mkdir(root_dir + '/catboost')
    if not exists(root_dir + '/nn'):
        os.mkdir(root_dir + '/nn')
    
    models_catboost, models_nn = {}, {}

    for filename in os.listdir(root_dir + '/catboost/'):
        if 'ipynb_checkpoints' in filename:
            continue
        models_catboost[filename] = joblib.load(root_dir + '/catboost/' + filename)

    for filename in os.listdir(root_dir + '/nn/'):
        if 'ipynb_checkpoints' in filename:
            continue
        models_nn[filename] = tf.keras.models.load_model(root_dir + '/nn/' + filename)

    if use_models == "NN":
        models = models_nn
    else:
        models = models_catboost

    models = sorted(models.items(), key=lambda x: x[0], reverse=False)
    models = {k: v for k, v in models}

    for model_name, _ in models.items():
        print(model_name + ' loaded!')

### create_models:

In [13]:
def create_models():
    if len(models.keys()) == 0 or start_over:
        if use_models == "NN":
            data = create_model_nn('NNAction')
        else:
            data = create_model_catboost('CatBoostAction')
        models[data[0]] = data[1]
        print(data[0] + ' model created!')

def create_model_nn(name):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=2))
    model.add(Dense(4, activation='softmax'))
    model.compile(optimizer='adam', loss=' categorical_crossentropy', metrics=['accuracy'])

    return name, model

def create_model_catboost(name):
    param = {
        "iterations": catboost_iterations,
        "depth": catboost_depth,
        'learning_rate': catboost_learning_rate,
        "one_hot_max_size": catboost_one_hot_max_size,
        "ctr_target_border_count": catboost_ctr_target_border_count,
        "model_size_reg": catboost_model_size_reg,
        "max_ctr_complexity": catboost_max_ctr_complexity,
        'l2_leaf_reg': catboost_l2_leaf_reg,
        'min_data_in_leaf': catboost_min_data_in_leaf,
        'random_strength': catboost_random_strength,
        "bootstrap_type": catboost_bootstrap_type,

        "objective": "MultiClass",
        "allow_const_label": True,
        "task_type": "GPU", 
        "has_time": True, 
        "class_names": possible_labels,
        "random_state": 420,
        "allow_writing_files": False,
        "boosting_type": "Plain",
    }
    if catboost_bootstrap_type == "Bayesian":
        param["bagging_temperature"] = catboost_bagging_temperature
    elif catboost_bootstrap_type == "Bernoulli":
        param["subsample"] = catboost_subsample

    model = CatBoostClassifier(**param)

    return name, model

### populate_coins_to_pairs:

In [14]:
def validateActiveTickerSymbol(ticker):
    if float(ticker['lastPrice']) < 0.0000001 or float(ticker['bidPrice']) < 0.0000001 or float(ticker['askPrice']) < 0.0000001:
        return False
    return True
    
def populate_coins_to_pairs():
    global avaiable_coins, avaiable_symbols_list
    avaiable_coins = {}
    avaiable_symbols_list = []

    balances = binance.get_account()['balances']
    tickers = binance.get_ticker() # All symbol info


    for balance in balances:
        coin = balance['asset']
        avaiable_coins[coin] = []

    for ticker in tickers:
        if not validateActiveTickerSymbol(ticker):
            continue
        symbol = ticker['symbol']

        found = False
        for balance in balances:
            coin = balance['asset']
            if isCoinPairMatching(coin, symbol):
              avaiable_coins[coin].append(symbol)
              found = True
        if found:
            avaiable_symbols_list.append(symbol)

def isCoinPairMatching(coin, symbol):
    if coin not in symbol:
        return False
    
    symbols = ["BTCUP", "BTCDOWN", "ADAUP", "ADADOWN", "ETHUP", "ETHDOWN", "DOTUP", "DOTDOWN", "TRXUP", "TRXDOWN", "LINKUP", "LINKDOWN", \
                "BNBUP", "BNBDOWN", "CRH"]
    if any(x in symbol for x in symbols):
        return False

    if coin == 'AMB':
        similars = ["CREAMBUSD", "BEAM"]
        if any(x in symbol for x in similars):
            return False 
    elif coin == 'AUD':
        similars = ["AUDIO"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'TRU':
        similars = ["ASTRUSDT", "USDTRUB", "DOT"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'TUSD':
        similars = ["TUSDT"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'GAL':
        similars = ["GALA"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'BTC':
        similars = ["BTCST"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'OG':
        similars = ["DOGE", "OGN"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'COS':
        similars = ["COCOS"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'RUB':
        similars = ["TRUBTC"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'SHIB':
        similars = ["SUSHIBTC", "SUSHIBUSD", "SUSHIBNB"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'FET':
        similars = ["ELFETH"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'YFI':
        similars = ["YFII"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'TRB':
        similars = ["ASTRBTC", "ASTRBUSD"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'BAR':
        similars = ["HBAR"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'SC':
        similars = ["SCRT"]
        if any(x in symbol for x in similars):
           return False
    elif coin == 'REP':
        similars = ["DREP"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'GO':
        similars = ["AERGO", "ALGO", "DEGO"]
        if any(x in symbol for x in similars):
           return False
    elif coin == 'AST':
        similars = ["ASTR"]
        if any(x in symbol for x in similars):
           return False
    elif coin == 'AST':
        similars = ["CRVETH", "SSVETH"]
        if any(x in symbol for x in similars):
           return False
    elif coin == 'ONE':
        similars = ["AIONETH"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'GLM':
        similars = ["GLMR"]
        if any(x in symbol for x in similars):
           return False
    elif coin == 'DAR':
        similars = ["ADARUB"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'VET':
        similars = ["CRVETH", "SSVETH"]
        if any(x in symbol for x in similars):
           return False
    elif coin == 'OXT':
        similars = ["MBOXTRY"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'ACA':
        similars = ["ALPACA"]
        if any(x in symbol for x in similars):
           return False
    elif coin == 'ONT':
        similars = ["FRONT"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'TCT':
        similars = ["BTCTUSD", "BTTCTRY", "BTCTRY"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'AMP':
        similars = ["RAMP"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'WBTC':
        similars = ["FLOWBTC"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'LPT':
        similars = ["SLPTRY"]
        if any(x in symbol for x in similars):
           return False
    elif coin == 'PHA':
        similars = ["ALPHA"]
        if any(x in symbol for x in similars):
           return False
    elif coin == 'AVA':
        similars = ["KAVA", "AVAX"]
        if any(x in symbol for x in similars):
           return False
    elif coin == 'MOB':
        similars = ["TOMO"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'ORN':
        similars = ["TORN"]
        if any(x in symbol for x in similars):
           return False
    elif coin == 'OM':
        similars = ["OMG", "ATOM", "COMP", "LOOM", "TOMO", "PROM"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'AR':
        similars = ["ARDR", "ARK", "ARPA", "BAR", "FARM", "HARD", "HBAR", "NEAR", "RARE", "DAR", "SPARTA"]
        if any(x in symbol for x in similars):
           return False
    elif coin == 'ATA':
        similars = ["DATA"]
        if any(x in symbol for x in similars):
           return False
    elif coin == 'ANT':
        similars = ["SANTOS"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'WIN':
        similars = ["WING"]
        if any(x in symbol for x in similars):
           return False
    elif coin == 'BUSD':
        similars = ["BNBUSDC", "TRBUSDT", "MOBUSDT"]
        if any(x in symbol for x in similars):
           return False
    elif coin == 'OP':
        similars = ["PEOPLE"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'FOR':
        similars = ["FORTH"]
        if any(x in symbol for x in similars):
            return False
    elif coin == 'CKB':
        similars = ["DOCK", "QUICK"]
        if any(x in symbol for x in similars):
           return False
    elif coin in ['BETH', 'BDOT', 'T']:
        return False

    return True

### initBinanceInfo:

In [15]:
def initBinanceInfo():
    global binance_symbols

    get_exchange_info = binance.get_exchange_info()
    for symbol in get_exchange_info['symbols']:
        if symbol['symbol'] in avaiable_symbols_list:
            stepSize = 0
            stepSizeInteger = 0
            tickSize = 0
            minNotional = 0
            for filter in symbol['filters']:
                if filter['filterType'] == 'LOT_SIZE':
                    stepSize = float(filter['stepSize'])
                    if filter['stepSize'] == '0.10000000':
                        stepSizeInteger = 1
                    elif filter['stepSize'] == '0.01000000':
                        stepSizeInteger = 2
                    elif filter['stepSize'] == '0.00100000':
                        stepSizeInteger = 3
                    elif filter['stepSize'] == '0.00010000':
                        stepSizeInteger = 4
                    elif filter['stepSize'] == '0.00001000':
                        stepSizeInteger = 5
                    elif filter['stepSize'] == '0.00000100':
                        stepSizeInteger = 6
                    elif filter['stepSize'] == '0.00000010':
                        stepSizeInteger = 7
                    elif filter['stepSize'] == '0.00000001':
                        stepSizeInteger = 8
                elif filter['filterType'] == 'PRICE_FILTER':
                    if filter['tickSize'] == '0.10000000':
                        tickSize = 1
                    elif filter['tickSize'] == '0.01000000':
                        tickSize = 2
                    elif filter['tickSize'] == '0.00100000':
                        tickSize = 3
                    elif filter['tickSize'] == '0.00010000':
                        tickSize = 4
                    elif filter['tickSize'] == '0.00001000':
                        tickSize = 5
                    elif filter['tickSize'] == '0.00000100':
                        tickSize = 6
                    elif filter['tickSize'] == '0.00000010':
                        tickSize = 7
                    elif filter['tickSize'] == '0.00000001':
                        tickSize = 8
                elif filter['filterType'] == 'MIN_NOTIONAL':
                    minNotional = float(filter['minNotional'])

            binance_symbols[symbol['symbol']] = {
                'stepSize': stepSize,
                'stepSizeInteger': stepSizeInteger,
                'tickSize': tickSize,
                'minNotional': minNotional,
            }

### init_meta:

In [16]:
root_dir = 'models' 
categorial_features = ['direction_', 'symbol', 'coin']

if is_google_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    
    root_dir = 'drive/MyDrive/tradebot'

if is_google_colab or is_google_cloud:
    !pip install catboost
    !pip install python-binance
    !pip install tensorflow

import os, psutil
from os.path import exists

from catboost import *
import joblib

import warnings
warnings.filterwarnings('ignore')

from binance import Client
from binance.enums import *

import sched, time
import pandas as pd

import datetime
import time
from threading import Thread, Lock
import numpy as np
import random

from keras.layers import Dense
from keras.models import Sequential

models = {}
categorial_features_indices = None
holding_coins = None
features_list = None
features_dtypes = None
avaiable_coins = None
avaiable_symbols_list = None

possible_labels = ['NONE']
for i in range(0, amount_of_symbols_per_action_sample):
    possible_labels.append(str(i))

binance_api_key = 'AoVFfn3JvetSRHpffstx9tg0Zlmzc6WHeAdVjVUnLfbzOTslBanPUMFa7bP4CqtU'
binance_api_secret = 'zNGuMcE2UcycFQeVhTi5o9psM6GjZHvg7gEcTu1f8pazQg42EAMgvma5G583wv4G'
binance = Client(binance_api_key, binance_api_secret)

##### :
epoch, part_epoch, total_part_epochs = 1, 1, 0
memory_data = []
memory_symbols = {}
predictions_saved = []
deleted_symbol_data = 0
print_trained_string = '---'
loss_string = '-'
model_training_lock = None
model_df_buffer_lock = None
handleTickers_lock = None
binance_symbols = {} #
dont_update_holding_coins = []
dont_update_holding_coins_buffer = {}
cancel_orders = False
temp_tickers = {}
do_predictions = True
saved_df_buffer = pd.DataFrame()
saved_y_buffer = []
init_time = None

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [25]:
populate_coins_to_pairs()
initBinanceInfo()
load_models()
create_models()

BinanceAPIException: APIError(code=-1021): Timestamp for this request is outside of the recvWindow.

## Functions

### Init:

In [ ]:
def init():
    global holding_coins, do_predictions, model_training_lock, model_df_buffer_lock, \
    epoch, part_epoch, total_part_epochs, memory_data, memory_symbols, predictions_saved, \
    deleted_symbol_data, print_trained_string, loss_string, saved_y_buffer, \
    dont_update_holding_coins, dont_update_holding_coins_buffer, \
    cancel_orders, temp_tickers, saved_df_buffer, \
    handleTickers_lock, init_time

    initBinanceInfo()

    epoch, part_epoch, total_part_epochs = 1, 1, 0
    memory_data = []
    memory_symbols = {}
    predictions_saved = []
    deleted_symbol_data = 0
    print_trained_string = '---'
    loss_string = '-'
    model_training_lock = None
    model_df_buffer_lock = None
    handleTickers_lock = None
    dont_update_holding_coins = []
    dont_update_holding_coins_buffer = {}
    cancel_orders = False
    temp_tickers = {}
    do_predictions = True
    saved_df_buffer = pd.DataFrame()
    saved_y_buffer = []
    init_time = time.time()

    holding_coins = {
        'DEFAULT': {
            'balance': 0,
            'balance_eur': 1,
            'price': 0,
            'counting_epochs': 0,
            'symbol': '',
            'date': time.strftime("%H:%M:%S")
        }
    }

    for model_name, model in models.items():
        if not model.is_fitted():
            do_predictions = False

    model_training_lock = Lock()
    model_df_buffer_lock = Lock()
    handleTickers_lock = Lock()

    print("""\
                        ._ o o
                        \_`-)|_
                      ,""       \ 
                    ,"  ## |   = ಠ. 
                  ," ##   ,-\__    `.
                ,"       /     `--._;)
              ,"     ## /
            ,"   ##    /
███████╗████████╗ ██████╗ ███╗   ██╗██╗  ██╗██████╗  ██████╗ ████████╗
██╔════╝╚══██╔══╝██╔═══██╗████╗  ██║██║ ██╔╝██╔══██╗██╔═══██╗╚══██╔══╝
███████╗   ██║   ██║   ██║██╔██╗ ██║█████╔╝ ██████╔╝██║   ██║   ██║   
╚════██║   ██║   ██║   ██║██║╚██╗██║██╔═██╗ ██╔══██╗██║   ██║   ██║   
███████║   ██║   ╚██████╔╝██║ ╚████║██║  ██╗██████╔╝╚██████╔╝   ██║   
╚══════╝   ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚═╝  ╚═╝╚═════╝  ╚═════╝    ╚═╝   v0.2
                                                                      
                    """)
    print()

### Runner:

In [ ]:
def runner():
    global schedule, epoch, part_epoch, total_part_epochs, init_time
    time_now = time.time()
    
    getTickers(part_epoch)

    if part_epoch == (seconds_inbetween / seconds_inbetween_pibeline) - 1:
        thread = Thread(target = updateBinanceBalances)
        thread.start()

    if seconds_inbetween / seconds_inbetween_pibeline <= part_epoch:
        processTickers()
        thread = Thread(target = handleTickers, args = (time_now, epoch, memory_symbols,))
        thread.start()
        epoch += 1
        part_epoch = 0
    part_epoch += 1
    total_part_epochs += 1
    # 459: 6 sec break..
    if total_part_epochs == 460:
        sleepy = (seconds_inbetween_pibeline * total_part_epochs) - (time.time() - init_time) - seconds_inbetween_pibeline + 2.5
        if sleepy < 0:
          sleepy = 0
        print('sleepy', sleepy)
        time.sleep(sleepy)
        init_time = time.time()
        total_part_epochs = 0
        return

    schedule.enter((seconds_inbetween_pibeline * total_part_epochs) - (time.time() - init_time) - seconds_inbetween_pibeline, 1, runner)
    schedule.run()

def getTickers(part_epoch):
  global temp_tickers

  try:
      tickers = binance.get_ticker() # All symbol info
  except:
      print('binance.get_ticker() FAILED 1/2.')
      try:
          tickers = binance.get_ticker() # All symbol info
      except:
          print('binance.get_ticker() FAILED 2/2.')
          return
  
  if part_epoch == 1:
      for symbol in tickers:
          if symbol['symbol'] not in avaiable_symbols_list:
              continue
          temp_tickers[symbol['symbol']] = {
              'symbol': symbol['symbol'],
              'bidPrice_low': float(symbol['bidPrice']),
              'bidPrice_high': float(symbol['bidPrice']),
              'askPrice_low': float(symbol['askPrice']),
              'askPrice_high': float(symbol['askPrice']),
              'volume': float(symbol['volume']),
              'quoteVolume': float(symbol['quoteVolume']),
              'tradeCount': int(symbol['count']),
              'lastPrice': float(symbol['lastPrice']),
          }
        
  else:
      for symbol in tickers:
          if symbol['symbol'] not in avaiable_symbols_list:
              continue
        
          new_bidPrice = float(symbol['bidPrice'])
          new_askPrice = float(symbol['askPrice'])
          
          temp_tickers[symbol['symbol']] = {
              'symbol': symbol['symbol'],
              'bidPrice_low': new_bidPrice if temp_tickers[symbol['symbol']]['bidPrice_low'] > new_bidPrice else temp_tickers[symbol['symbol']]['bidPrice_low'],
              'bidPrice_high': new_bidPrice if temp_tickers[symbol['symbol']]['bidPrice_high'] < new_bidPrice else temp_tickers[symbol['symbol']]['bidPrice_high'],
              'askPrice_low': new_askPrice if temp_tickers[symbol['symbol']]['askPrice_low'] > new_askPrice else temp_tickers[symbol['symbol']]['askPrice_low'],
              'askPrice_high': new_askPrice if temp_tickers[symbol['symbol']]['askPrice_high'] < new_askPrice else temp_tickers[symbol['symbol']]['askPrice_high'],
              'volume': float(symbol['volume']),
              'quoteVolume': float(symbol['quoteVolume']),
              'tradeCount': int(symbol['count']),
              'lastPrice': float(symbol['lastPrice']),
          }
        
          # if new_bidPrice == 0 or new_askPrice == 0:
          #     print(symbol)
          #     del temp_tickers[symbol['symbol']]
          #     del memory_symbols[symbol['symbol']]
          #     avaiable_symbols_list.remove(symbol['symbol'])
          #     print(symbol['symbol'], 'removed.', symbol['bidPrice'], symbol['askPrice'])

def processTickers():
    global memory_symbols
    for coin in temp_tickers.values():
        if coin['symbol'] not in memory_symbols:
            memory_symbols[coin['symbol']] = {
                'bidPrice_low': [],
                'bidPrice_high': [],
                'askPrice_low': [],
                'askPrice_high': [],
                'volume': [],
                'quoteVolume': [],
                'tradeCount': [],
                'lastPrice': [],
            }
        memory_symbols[coin['symbol']]['bidPrice_low'].append(coin['bidPrice_low'])
        memory_symbols[coin['symbol']]['bidPrice_high'].append(coin['bidPrice_high'])
        memory_symbols[coin['symbol']]['askPrice_low'].append(coin['askPrice_low'])
        memory_symbols[coin['symbol']]['askPrice_high'].append(coin['askPrice_high'])
        memory_symbols[coin['symbol']]['volume'].append(coin['volume'])
        memory_symbols[coin['symbol']]['quoteVolume'].append(coin['quoteVolume'])
        memory_symbols[coin['symbol']]['tradeCount'].append(coin['tradeCount'])
        memory_symbols[coin['symbol']]['lastPrice'].append(coin['lastPrice'])


### MAIN Thread:

In [ ]:
def handleTickers(time_now_big, epoch, memory_symbols):
    global memory_data, deleted_symbol_data, print_trained_string, do_predictions, do_predictions, \
    predictions_saved, handleTickers_Lock, categorial_features_indices, features_dtypes, features_list

    # Get list of symbols to check:
    use_symbols = []
    for coin in holding_coins.keys():
        use_symbols += avaiable_coins[coin]

    memory_data_symbol = None
    predictions_byModel = [
        {},
        '0',
    ]
    
    handleTickers_lock.acquire() #
    print('--------  Epoch: ', epoch, '  [' + str(time.strftime("%H:%M:%S")) + ']')
    print()

    # Proccess coin data
    proccessTickers_time = ''
    if epoch > 1:
        time_big_itcp = time.time() - time_now_big
        time_now = time.time()
        memory_data_symbol = proccessTickers(epoch, memory_symbols, True, use_symbols)
        proccessTickers_time = str(round(time_big_itcp, 2)) + ' + ' + str(round(time.time() - time_now, 2))

    if epoch > pred_epoch:
        # Process Action Data
        time_now = time.time()
        saved = processActionData(epoch, memory_data_symbol, True)
        proccessTickers_time += ' + ' + str(round(time.time() - time_now, 2))

        ## Init Set features_dtypes & categorial_features_indices ##
        if epoch == pred_epoch + 1:
            features_list = list(saved[list(holding_coins.keys())[0]].keys())
            features_dtypes = {}
            categorial_features_indices = []
            i = 0
            for feature in features_list:
                if any(x in feature for x in categorial_features):
                    categorial_features_indices.append(i)
                i += 1
                if 'coin' in feature or 'symbol' in feature or 'direction_' in feature:
                    features_dtypes[feature] = 'object'
                else:
                    features_dtypes[feature] = 'float32'
        ####

        if do_predictions:
            # Do Predictions:
            predictions_byModel = proccessPredictions(epoch, saved)


    # Balances and euro:
    print_eur_string = convert_wallet_to_euro(holding_coins, memory_symbols, epoch)
    print('[ ' + print_eur_string + ' ]   _ Balances:')
    for coin, inner in holding_coins.items():
        print(coin, str(round(inner['balance'], 4)), '', str(round(inner['balance_eur'], 2)) + ' €', '   |  Price: ' + str(inner['price']) + ' ' + inner['symbol'] + '  | Epoch: ' + str(inner['counting_epochs']) + '       [' + inner['date'] + ']')
    print()
    
    # Trade coins:
    if do_predictions and epoch > interval_columns + pred_epoch + 50:
        processTrading(predictions_byModel[0], epoch, memory_symbols)
    #

    # Now build data for the rest of the coins and symbols:
    if epoch > 1:
        memory_data_symbol = proccessTickers(epoch, memory_symbols, False, None)

        # Start updating symbol priority list into avaiable_coins:
        thread = Thread(target = updateSymbolPriorityList, args = (memory_data_symbol,))
        thread.start()

    if epoch > pred_epoch:
        memory_data.append(processActionData(epoch, memory_data_symbol, False))
    #
    
    if epoch % 20 == 0 and epoch > pred_epoch:
        for model in models.values():
            if model.is_fitted():
                do_predictions = True
            else:
                do_predictions = False

    
    if epoch > pred_epoch:
        predictions_saved.append(predictions_byModel[0])

        # Do Validation on Past predictions:
        training_list, y_train, validated_list_targets = validatePredictions(epoch, memory_symbols)
        if do_predictions:
            validatePastPredictions(validated_list_targets)
            
        if epoch == pred_epoch + interval_columns - 20:
            # Do warmup:
            print('Will try to do warmup now...')
            thread = Thread(target = do_warmup, args = (training_list, y_train,))
            thread.start()

        elif epoch > pred_epoch + interval_columns:
            # Do training:
            thread = Thread(target = trainModels, args = (epoch, training_list, y_train,))
            thread.start()


    # Cleanup:
    if epoch > interval_columns:
        for k, value in memory_symbols.items():
            del value['bidPrice_high'][0]
            del value['bidPrice_low'][0]
            del value['askPrice_high'][0]
            del value['askPrice_low'][0]
            del value['volume'][0]
            del value['quoteVolume'][0]
            del value['tradeCount'][0]
            del value['lastPrice'][0]
        deleted_symbol_data += 1
    if len(memory_data) == pred_epoch:
        del memory_data[0]
        del predictions_saved[0]
    ##
    print('---------------- ', total_part_epochs, '[', round(time.time()-time_now_big, 2), 's ==', 't: ' + proccessTickers_time, '| p:', predictions_byModel[1], ']', \
          ' || ', print_trained_string, '|  Loss:', loss_string, ' |  RAM:', round(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2), 'mb')
    print()
    print()
    
    handleTickers_lock.release()

### proccessTickers():

In [ ]:
def proccessTickers(epoch, memory_symbols, only_use_holding_symbols, use_symbols):
    memory_data_symbol = {}

    for symbol, inner in memory_symbols.items():
        if only_use_holding_symbols and symbol not in use_symbols:
            continue

        dat = {
              'symbol': symbol,                               # categorical
              # 'weekday': str(currentime.weekday() + 1),     # categorical
              # 'hour': str(currentime.hour + 1),             # categorical
        }

        temp_bidPrice_low = inner['bidPrice_low'][epoch - 1 - deleted_symbol_data]
        temp_bidPrice_high = inner['bidPrice_high'][epoch - 1 - deleted_symbol_data]
        temp_askPrice_low = inner['askPrice_low'][epoch - 1 - deleted_symbol_data]
        temp_askPrice_high = inner['askPrice_high'][epoch - 1 - deleted_symbol_data]
        temp_volume = inner['volume'][epoch - 1 - deleted_symbol_data]
        if temp_volume == 0:
            temp_volume = 0.0000001
        temp_quoteVolume = inner['quoteVolume'][epoch - 1 - deleted_symbol_data]
        if temp_quoteVolume == 0:
            temp_quoteVolume = 0.0000001
        temp_tradeCount = inner['tradeCount'][epoch - 1 - deleted_symbol_data]
        if temp_tradeCount == 0:
            temp_tradeCount = 0.0000001
        temp_lastPrice = inner['lastPrice'][epoch - 1 - deleted_symbol_data]
        if temp_lastPrice == 0:
            temp_lastPrice = 0.0000001

        seconds_ago = seconds_inbetween

        for i in range(1, interval_columns + 1):
            seconds_ago_string = str(seconds_ago)
            if (epoch > i):
                index = epoch - i - 1 - deleted_symbol_data
                
                bidPrice_low = inner['bidPrice_low'][index] # Reverse + Pick i element.
                if bidPrice_low == 0:
                    dat['bidPrice_low_' + seconds_ago_string + '_secAgo'] = 0
                else:
                    dat['bidPrice_low_' + seconds_ago_string + '_secAgo'] = (temp_bidPrice_low - bidPrice_low) / bidPrice_low

                bidPrice_high = inner['bidPrice_high'][index] # Reverse + Pick i element.
                if bidPrice_high == 0:
                    dat['bidPrice_high_' + seconds_ago_string + '_secAgo'] = 0
                else:
                    dat['bidPrice_high_' + seconds_ago_string + '_secAgo'] = (temp_bidPrice_high - bidPrice_high) / bidPrice_high

                askPrice_low = inner['askPrice_low'][index] # Reverse + Pick i element.
                if askPrice_low == 0:
                    dat['askPrice_low_' + seconds_ago_string + '_secAgo'] = 0
                else:
                    dat['askPrice_low_' + seconds_ago_string + '_secAgo'] = (temp_askPrice_low - askPrice_low) / askPrice_low

                askPrice_high = inner['askPrice_high'][index] # Reverse + Pick i element.
                if askPrice_high == 0:
                    dat['askPrice_high_' + seconds_ago_string + '_secAgo'] = 0
                else:
                    dat['askPrice_high_' + seconds_ago_string + '_secAgo'] = (temp_askPrice_high - askPrice_high) / askPrice_high

                volume = inner['volume'][index] # Reverse + Pick i element.
                if volume == 0:
                    volume = 0.0000001
                dat['volume_' + seconds_ago_string + '_secAgo'] = (temp_volume - volume) / volume

                quoteVolume = inner['quoteVolume'][index] # Reverse + Pick i element.
                if quoteVolume == 0:
                    quoteVolume = 0.0000001
                dat['quoteVolume_' + seconds_ago_string + '_secAgo'] = (temp_quoteVolume - quoteVolume) / quoteVolume

                tradeCount = inner['tradeCount'][index] # Reverse + Pick i element.
                if tradeCount == 0:
                    tradeCount = 0.0000001
                dat['tradeCount_' + seconds_ago_string + '_secAgo'] = (temp_tradeCount - tradeCount) / tradeCount

                lastPrice = inner['lastPrice'][index] # Reverse + Pick i element.
                if lastPrice == 0:
                    lastPrice = 0.0000001
                dat['lastPrice_' + seconds_ago_string + '_secAgo'] = (temp_lastPrice - lastPrice) / lastPrice
            else:
                dat['bidPrice_low_' + seconds_ago_string + '_secAgo'] = 0
                dat['bidPrice_high_' + seconds_ago_string + '_secAgo'] = 0
                dat['askPrice_low_' + seconds_ago_string + '_secAgo'] = 0
                dat['askPrice_high_' + seconds_ago_string + '_secAgo'] = 0
                dat['volume_' + seconds_ago_string + '_secAgo'] = 0
                dat['quoteVolume_' + seconds_ago_string + '_secAgo'] = 0
                dat['tradeCount_' + seconds_ago_string + '_secAgo'] = 0
                dat['lastPrice_' + seconds_ago_string + '_secAgo'] = 0

            direction = 'BIGGEST_DOWN'
            lastPrice = dat['lastPrice_' + seconds_ago_string + '_secAgo']
            if lastPrice > 3:
              direction = 'BIGGEST_UP'
            elif lastPrice > 2:
              direction = 'BIGGER_UP'
            elif lastPrice > 1:
              direction = 'BIG_UP'
            elif lastPrice > 0.5:
              direction = 'HALF_UP'
            elif lastPrice > binary_target_minimum_increase_min * 2:
              direction = 'UP_UP'
            elif lastPrice > binary_target_minimum_increase_min:
              direction = 'UP'
            elif lastPrice > 0:
              direction = 'NEUTRAL'
            elif lastPrice == 0:
              direction = 'NULL'
            elif lastPrice > -binary_target_minimum_increase_min:
              direction = 'DOWN'
            elif lastPrice > -binary_target_minimum_increase_min * 2:
              direction = 'DOWN_DOWN'
            elif lastPrice > -0.5:
              direction = 'DOWNER'
            elif lastPrice > -1:
              direction = 'HALF_DOWN'
            elif lastPrice > -2:
              direction = 'BIG_DOWN'
            elif lastPrice > -3:
              direction = 'BIGGER_DOWN'
            dat['direction_' + seconds_ago_string + '_secAgo'] = direction

            seconds_ago += seconds_inbetween
            

        memory_data_symbol[symbol] = dat

    return memory_data_symbol

### processActionData():

In [ ]:
def processActionData(epoch, memory_data_symbol, only_use_holding_symbols):
    global holding_coins, memory_data, categorial_features_indices, features_list, features_dtypes, saved_df_buffer

    saved = {}
    if only_use_holding_symbols:
        for coin, inner in holding_coins.items():
            counting_epochs = inner['counting_epochs']
            if counting_epochs > pred_epoch:
                counting_epochs = pred_epoch
            saved[coin] = getActionSampleRow(coin, memory_data_symbol, counting_epochs)
    else:
        for coin in avaiable_coins.keys():
            counting_epochs = random.randint(0, pred_epoch)
            saved[coin] = getActionSampleRow(coin, memory_data_symbol, counting_epochs)

    return saved


def getActionSampleRow(coin, memory_data_symbol, counting_epochs):
    minimum_threshold = binary_target_minimum_increase_min
    if counting_epochs < pred_epoch:
        minimum_threshold = binary_target_minimum_increase_max - (counting_epochs * (binary_target_minimum_increase_max / pred_epoch))
 
    data = {
      'coin': coin,
      'minimum_threshold': minimum_threshold, 
    }

    i = 0
    for symbol in avaiable_coins[coin]:
        if i > amount_of_symbols_per_action_sample - 1:
            break

        if symbol.startswith(coin):
            for attr, value in memory_data_symbol[symbol].items():
                if 'Price_' in attr or 'olume_' in attr or 'tradeCount_' in attr:
                    if value != 0:
                        data[str(i) + attr] = value * -1
                    else:
                        data[str(i) + attr] = value
                elif 'direction_' in attr:
                    data[str(i) + attr] = changeDirection(value)
                else:
                    data[str(i) + attr] = value
        else:
            for attr, value in memory_data_symbol[symbol].items():
                data[str(i) + attr] = value

        i += 1

    for a in range (i, amount_of_symbols_per_action_sample):
        for attr in memory_data_symbol[debug_symbol].keys():
            data[str(a) + attr] = '0'

    return data

def changeDirection(value):
    if value == 'BIGGEST_DOWN':
        return 'BIGGEST_UP'
    if value == 'BIGGER_DOWN':
        return 'BIGGER_UP'
    if value == 'BIG_DOWN':
        return 'BIG_UP'
    if value == 'HALF_DOWN':
        return 'HALF_UP'
    if value == 'DOWNER':
        return 'UP_UP'
    if value == 'DOWN_DOWN':
        return 'UP_UP'
    if value == 'DOWN':
        return 'UP'
    if value == 'NEUTRAL':
        return 'DOWN'
    if value == 'UP':
        return 'DOWN'
    if value == 'UP_UP':
        return 'DOWN_DOWN'
    if value == 'HALF_UP':
        return 'HALF_DOWN'
    if value == 'BIG_UP':
        return 'BIG_DOWN'
    if value == 'BIGGER_UP':
        return 'BIGGER_DOWN'
    if value == 'BIGGEST_UP':
        return 'BIGGEST_DOWN'
    return value

### updateBinanceBalances + updateSymbolPriorityList:

In [ ]:
def updateBinanceBalances():
    global holding_coins, dont_update_holding_coins, cancel_orders

    if cancel_orders:
        cancel_orders = False
        orders = binance.get_open_orders()
        for order in orders:
            print('cancel_order:', binance.cancel_order(
                symbol = order['symbol'],
                orderId = order['orderId']))

    balances = binance.get_account()['balances']
    found_assets = []
    for balance in balances:
        balance_number = float(balance['free'])
        if balance_number > 0.0001:
            if balance['asset'] not in holding_coins.keys():
                if 'DEFAULT' in holding_coins.keys():
                    holding_coins[balance['asset']] = holding_coins['DEFAULT'].copy()
                else:
                    continue
            if balance['asset'] == 'EUR':
                balance_number -= real_wallet_dont_use_euro
                if balance_number < 1:
                    continue
            if balance['asset'] == 'BNB':
                balance_number -= 0.01  # 2.76eur 15.09.22
            if balance_number < 0.0001:
                continue
            if balance['asset'] not in dont_update_holding_coins:
                holding_coins[balance['asset']]['balance'] = balance_number
            found_assets.append(balance['asset'])
    for coin in list(holding_coins.keys()):
        if coin not in found_assets or holding_coins[coin]['balance_eur'] < 1:
            if coin in dont_update_holding_coins:
                dont_update_holding_coins.remove(coin)
                del dont_update_holding_coins_buffer[coin]
            del holding_coins[coin]

In [ ]:
def updateSymbolPriorityList(memory_data_symbol):
    global avaiable_coins

    symbol_changes = {}
    for symbol, details in memory_data_symbol.items():
        symbol_changes[symbol] = 0
        symbol_changes[symbol] += getPositiveChange(details['bidPrice_high_' + str(seconds_inbetween) + '_secAgo'])
        symbol_changes[symbol] += getPositiveChange(details['askPrice_high_' + str(seconds_inbetween) + '_secAgo'])
        symbol_changes[symbol] += getPositiveChange(details['volume_' + str(seconds_inbetween) + '_secAgo'])
        symbol_changes[symbol] += getPositiveChange(details['quoteVolume_' + str(seconds_inbetween) + '_secAgo'])
        symbol_changes[symbol] += getPositiveChange(details['tradeCount_' + str(seconds_inbetween) + '_secAgo'])
        symbol_changes[symbol] += getPositiveChange(details['lastPrice_' + str(seconds_inbetween) + '_secAgo'])
    symbol_changes = list(dict(sorted(symbol_changes.items(), key=lambda item: item[1], reverse = True)).keys())

    new_avaiable_coins = {}
    for coin, symbols in avaiable_coins.items():
        new_avaiable_coins[coin] = []
        for symbol in symbol_changes:
            if symbol in symbols:
                new_avaiable_coins[coin].append(symbol)

    avaiable_coins = new_avaiable_coins


def getPositiveChange(number):
    if number < 0:
        return number * -1
    else:
        return number

### proccessPredictions():

In [ ]:
def runModelPredictions(predictions_byModel, model_name, model, df):
  
  results = model.predict_proba(df)

  for i in range(len(results)):
      coin = df['coin'][i]
      predictions_byModel[model_name][coin] = {}

      sorted_array = np.argsort(results[i])
      
      a = -1
      found = 0
      for result in results[i]:
          if found > 1:
              break
          a += 1

          position = None
          if a == sorted_array[-1]:
              position = 0
          elif a == sorted_array[-2]:
              position = 1
          else:
              continue

          found += 1
          label = possible_labels[a]
          symbol = 'NONE'

          if label != 'NONE':
              symbol = df[label + 'symbol'][i]
              # if symbol == '0':
              #     continue
          
          predictions_byModel[model_name][coin][position] = {
              'symbol': symbol,
              'value': result,
          }

  return predictions_byModel

def proccessPredictions(epoch, saved):
    time_now = time.time()
    list_data = []
    predictions_byModel = {}
    
    for _, sample in saved.items():
        list_data.insert(0, list(sample.values()))

    df = pd.DataFrame(list_data, columns = features_list)

    for model_name, model in models.items():
        predictions_byModel[model_name] = {}
        predictions_byModel = runModelPredictions(predictions_byModel, model_name, model, df)

    pred_time = str(round(time.time() - time_now, 2))

    return predictions_byModel, pred_time

### validatePredictions:

In [ ]:
def validatePredictions(current_epoch, memory_symbols):
    global memory_data, predictions_saved
    
    list_list = []
    list_data = []
    y_train = []
    validated_list = {}
    validated_list_targets = {}

    for symbol, inner in memory_symbols.items():
        old_lastPrice = inner['lastPrice'][0]
        new_lastPrice = inner['lastPrice'][current_epoch - 1 - deleted_symbol_data]

        validated_list[symbol] = (new_lastPrice - old_lastPrice) / old_lastPrice
        
    for coin, sample in memory_data[0].items():
        biggest_symbol = {
            'result': 'NONE',
            'name': 'NONE',
            'reg_target': sample['minimum_threshold'],
        }

        for i in range(0, amount_of_symbols_per_action_sample):
            symbol = sample[str(i) + 'symbol']
            if symbol is not '0':
                reg_target = validated_list[symbol]
                if symbol.startswith(coin):
                    reg_target *= -1
                if reg_target > biggest_symbol['reg_target']:
                    biggest_symbol['result'] = str(i)
                    biggest_symbol['name'] = symbol
                    biggest_symbol['reg_target'] = reg_target

        list_data.append(sample)
        y_train.append(biggest_symbol['result'])
        validated_list_targets[coin] = biggest_symbol['name']

    return list_data, y_train, validated_list_targets

In [ ]:
def validatePastPredictions(validated_list_targets):
    global loss_string

    loss_string = ''
    validated_binary_result = {}

    for model_name, inner in predictions_saved[0].items():
        validated_binary_result[model_name] = {}

        for coin, detail in inner.items():
            pred_0 = detail[0]
            pred_1 = detail[1]
            target = validated_list_targets[coin]

            if pred_0['symbol'] != target:
                validated_binary_result[model_name][coin] = pred_0['value']
            elif pred_1['symbol'] != target:
                validated_binary_result[model_name][coin] = pred_1['value']

        # Calculate Loss:
        losses = 0
        for coin, value in validated_binary_result[model_name].items():
            losses += value
        
        losses /= len(validated_binary_result[model_name].keys())

        losses_agregated[model_name].append(losses)

        if len(losses_agregated[model_name]) > aggregate_loss_history_epochs:
            del losses_agregated[model_name][0]
        
        median = round(sum(losses_agregated[model_name]) / len(losses_agregated[model_name]), 5)

        loss_string += str(median) + '  '
        #


losses_agregated = {}
for model_name in models.keys():
    losses_agregated[model_name] = []

### convert_wallet_to_euro():

In [ ]:
def convert_wallet_to_euro(holding_coins, memory_symbols, epoch):
    global dont_update_holding_coins, dont_update_holding_coins_buffer

    euro = 0

    for coin, data in holding_coins.items():
        found = False
        if coin == 'EUR':
            euro += data['balance']
            holding_coins[coin]['balance_eur'] = data['balance']
            found = True
            continue

        if coin == 'DEFAULT':
            continue
        
        for symbol, inner in memory_symbols.items():
            if symbol in avaiable_coins[coin] and 'EUR' in symbol:
                lastPrice = inner['lastPrice'][epoch - 1 - deleted_symbol_data]
                balance = 0
                if symbol.endswith(coin):
                    balance = data['balance'] / lastPrice
                else:
                    balance = data['balance'] * lastPrice
                    
                euro += balance
                holding_coins[coin]['balance_eur'] = balance
                found = True
                break
        else:
            for symbol in avaiable_coins[coin]:
                lastPrice = memory_symbols[symbol]['lastPrice'][epoch - 1 - deleted_symbol_data]
                similars = ["BTC", "ETH", "BNB", "USDT", "BUSD"]
                if any(x in symbol for x in similars):
                    new_currency = 0
                    if symbol.endswith(coin):
                        new_currency = data['balance'] / lastPrice
                    else:
                        new_currency = data['balance'] * lastPrice
                    new_coin = symbol.replace(coin, '')
                    
                    # print(symbol, coin, new_coin) 
                    for symbol2 in memory_symbols.keys():
                        if symbol2 in avaiable_coins[new_coin] and 'EUR' in symbol2:
                            lastPrice = memory_symbols[symbol2]['lastPrice'][epoch - 1 - deleted_symbol_data]
                            balance = 0
                            if symbol2.endswith(coin):
                                balance = new_currency / lastPrice
                            else:
                                balance = new_currency * lastPrice
                                
                            euro += balance
                            holding_coins[coin]['balance_eur'] = balance
                            found = True
                            break
                    else:
                        continue
                    break

        if found == False:            
          print('ALEED!!: convert_wallet_to_euro() | Couldnt resolve coin for trading euro:', coin)

    return str(round(euro, 2)) + ' €'

### processTrading:

In [ ]:
def processTrading(predictions_byModel, current_epoch, memory_symbols):
    global holding_coins, dont_update_holding_coins, dont_update_holding_coins_buffer, cancel_orders, debug_symbol

    print('_TRADING:')
    for model_name, inner in predictions_byModel.items():
        for coin, details in inner.items():
            if coin in holding_coins.keys():
                print(coin, details[0]['symbol'], str(round(details[0]['value'] * 100, 2)) + '%')
                print(coin, details[1]['symbol'], str(round(details[1]['value'] * 100, 2)) + '%')
    print()


    for coin in list(holding_coins.keys()):
        symbol = predictions_byModel[model_name_for_trading][coin][0]['symbol']
        if symbol == 'NONE':
            if holding_coins[coin]['counting_epochs'] > 99:
                symbol = predictions_byModel[model_name_for_trading][coin][1]['symbol']
                print('Will trade ' + coin + ' now because counting_epochs > 99.')
            else:
                continue
        elif (predictions_byModel[model_name_for_trading][coin][1]['symbol'] == "NONE" \
              and predictions_byModel[model_name_for_trading][coin][0]['value'] < minimum_prediction_prob_trade_NextIsNone) or \
              predictions_byModel[model_name_for_trading][coin][0]['value'] < minimum_prediction_prob_trade_NextIsNotNone:
            continue
        if symbol == '0':
            print(predictions_byModel[model_name_for_trading][coin])
            continue

        buying_coin = symbol.replace(coin, '')

        buying_quantity = None
        price = None

        if symbol.startswith(coin):
            bidPrice_low = memory_symbols[symbol]['bidPrice_low'][current_epoch - 1 - deleted_symbol_data]
            price = bidPrice_low
            buying_quantity = holding_coins[coin]['balance'] * price
        else:
            askPrice_high = memory_symbols[symbol]['askPrice_high'][current_epoch - 1 - deleted_symbol_data]
            price = askPrice_high
            buying_quantity = holding_coins[coin]['balance'] / price

        price = round(price, binance_symbols[symbol]['tickSize'])
        buying_quantity = round(buying_quantity, 7)

        trade = '--> Attempt Trading ' + str(holding_coins[coin]['balance']) + ' ' + coin + ' to ' + str(buying_quantity) + ' ' + buying_coin + \
                ' for ' + str(price) + '        [' + time.strftime("%H:%M:%S") + ']'
        print(trade)

        #########################################################
        #########################################################
        #########################################################
        
        if do_real_money_trade:
              
            side = SIDE_BUY
            order_quantity = buying_quantity
            order_notional = holding_coins[coin]['balance']
            if symbol.endswith(buying_coin):
                side = SIDE_SELL
                order_quantity = holding_coins[coin]['balance']
                order_notional = buying_quantity

            order_quantity = round(order_quantity, binance_symbols[symbol]['stepSizeInteger'])

            if binance_symbols[symbol]['minNotional'] >= order_notional:
                print('minNotional err', coin, binance_symbols[symbol]['minNotional'], order_quantity)
                del holding_coins[coin]
                if coin in dont_update_holding_coins:
                    dont_update_holding_coins.remove(coin)
                    del dont_update_holding_coins_buffer[coin]
                print(coin + ' deleted from trading wallet.')
                continue

            limit_price = format(price, '.' + str(binance_symbols[symbol]['tickSize']) + 'f')
            order_quantity = format(order_quantity, '.' + str(binance_symbols[symbol]['stepSizeInteger']) + 'f')

            print(symbol, side, order_quantity, limit_price)
        
            success = False
            try:
                cancel_orders = True
                order = binance.create_order(
                    symbol=symbol,
                    side=side,
                    type=ORDER_TYPE_LIMIT,
                    timeInForce=TIME_IN_FORCE_GTC,
                    quantity=order_quantity,
                    price=limit_price)
                print(order)
                success = True
            except Exception as e:
                print('create_order err:', e.message)
                if 'insufficient balance' in e.message or 'MIN_NOTIONAL' in e.message:
                    if coin not in dont_update_holding_coins:
                        if coin not in dont_update_holding_coins_buffer.keys():
                            dont_update_holding_coins_buffer[coin] = 1
                        else:
                            dont_update_holding_coins_buffer[coin] += 1
                            if dont_update_holding_coins_buffer[coin] > 2:
                                dont_update_holding_coins.append(coin)
                    else:
                        holding_coins[coin]['balance'] -= binance_symbols[symbol]['stepSize']
            print()
            ###
            ########################################################

            if success:
                balance = buying_quantity
                if buying_coin in holding_coins.keys():
                    balance += holding_coins[buying_coin]['balance']

                # Set the new acquired coin:
                holding_coins[buying_coin] = {
                    'balance': balance,
                    'balance_eur': 1, 
                    'price': limit_price,
                    'counting_epochs': 0,
                    'symbol': symbol, 
                    'date': time.strftime('%H:%M:%S')
                }

                debug_symbol = symbol

                if coin in dont_update_holding_coins:
                    dont_update_holding_coins.remove(coin)
                    del dont_update_holding_coins_buffer[coin]

    for coin in holding_coins.keys():
        holding_coins[coin]['counting_epochs'] += 1

### trainModels:

In [ ]:
def do_warmup(training_list, y_train):
    time_now = time.time()
    model_name, model = create_model('CatBoostAction')
    df = pd.DataFrame(training_list)
    model.fit(df, 
                y_train, 
                cat_features = categorial_features_indices, 
                verbose = False)
    print('Did warmup:', time.time() - time_now)

In [ ]:
def trainModels(epoch, training_list, y_train):
    global print_trained_string, saved_df_buffer, saved_y_buffer

    time_now = time.time()

    df = pd.DataFrame(training_list)

    if do_gather_dataset:
        model_df_buffer_lock.acquire()
        saved_df_buffer = pd.concat([saved_df_buffer, df])
        saved_y_buffer += y_train
        model_df_buffer_lock.release()

    time_df = time.time() - time_now

    model_training_lock.acquire()
    time_now = time.time()

    if do_training:
        for model_name, model in models.items():
            model.fit(df, 
                y_train, 
                cat_features = categorial_features_indices, 
                verbose = False)

    time_train = time.time() - time_now

    model_training_lock.release()

    if epoch % 10 == 0:
        save() # Save models

    print_trained_string = '[Trained ' + str(len(y_train)) + ' [' + str(round(time_df, 2)) + ' + ' + str(round(time_train, 2)) + ' s]]'

### save:

In [ ]:
def save():
    global saved_df_buffer, saved_y_buffer
    time_now = time.time()

    if do_training:
        if use_models == "NN":
            for model_name, model in models.items():
                model.save(root_dir + '/catboost/' + model_name)
        else:
            for model_name, model in models.items():
                joblib.dump(model, root_dir + '/catboost/' + model_name)

    if do_gather_dataset:
        if model_df_buffer_lock.locked():
            model_df_buffer_lock.acquire() #
            saved_df_buffer = pd.DataFrame()
            saved_y_buffer = []
            model_df_buffer_lock.release() #
            print('Aborted save df.', round(time.time() - time_now, 2), 's')
        else:
            time_string = str(round(time.time()))
            model_df_buffer_lock.acquire() #
            df_import = saved_df_buffer
            saved_df_buffer = pd.DataFrame()
            y_import = saved_y_buffer
            saved_y_buffer = []
            model_df_buffer_lock.release() #
            df_import.to_csv(root_dir + '/' + datasets_dir + '/' + time_string + '.csv', index = False)
            joblib.dump(y_import, root_dir + '/' + datasets_dir + '/' + time_string + '.csv.y')
            print('Saved df to', datasets_dir, '. ', round(time.time() - time_now, 2), 's', '', df_import.shape[0], len(y_import))

        print()

## Main

In [ ]:
init()
schedule = sched.scheduler(time.time, time.sleep)
while(1):
    runner()
    print('Next episode')


# TODO:
# Look into Time Series Split (skforecast)
# Add volatility indicator per coin/symbol 
#